<a href="https://colab.research.google.com/github/Portman123/Fallout-Hotels/blob/main/hotel%20location%20generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import requests
import numpy as np


In [ ]:
hoteldf = pd.DataFrame( data ={'name': ['Helios Apartments', 'George Hotel Amesbury Stonehenge ','Dakota Manchester'], 'stars': [1,2,3]})
hoteldf = hoteldf.assign(longitude = np.NaN )
hoteldf = hoteldf.assign(latitude = np.NaN)

In [ ]:
def make_request(locname,size):
  headers = {
    "apikey": "63a22cb0-4f81-11ec-817f-1761fa8c301b"}

  params = {"text": locname, "size":size}

  response = requests.get('https://app.geocodeapi.io/api/v1/search', headers=headers, params=params);
  return response

In [ ]:
hoteldf

,name,stars,longitude,latitude
0,Helios Apartments,1,NaN,NaN
1,George Hotel Amesbury Stonehenge,2,NaN,NaN
2,Dakota Manchester,3,NaN,NaN


In [ ]:
for i in range(len(hoteldf)):
  location = hoteldf.at[i,'name']
  json = make_request(location)
  try:
      json.get('features')[0].get('geometry').get('coordinates')
  except (KeyError, IndexError ):
      print('cannot get -' + location)
  else:
    lon, lat = json.get('features')[0].get('geometry').get('coordinates')
    hoteldf.at[i,'longitude'] = lon
    hoteldf.at[i,'latitude'] = lat


TypeError: ignored

In [ ]:
hoteldf

# Generate Hotels

## generate cities

In [ ]:
cities_df = pd.read_csv("https://raw.githubusercontent.com/Portman123/Fallout-Hotels/main/worldcities.csv")
cities_df

,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id
0,Tokyo,Tokyo,35.6897,139.6922,Japan,JP,JPN,Tōkyō,primary,37977000.0,1392685764
1,Jakarta,Jakarta,-6.2146,106.8451,Indonesia,ID,IDN,Jakarta,primary,34540000.0,1360771077
2,Delhi,Delhi,28.6600,77.2300,India,IN,IND,Delhi,admin,29617000.0,1356872604
3,Mumbai,Mumbai,18.9667,72.8333,India,IN,IND,Mahārāshtra,admin,23355000.0,1356226629
4,Manila,Manila,14.6000,120.9833,Philippines,PH,PHL,Manila,primary,23088000.0,1608618140
...,...,...,...,...,...,...,...,...,...,...,...
40996,Tukchi,Tukchi,57.3670,139.5000,Russia,RU,RUS,Khabarovskiy Kray,NaN,10.0,1643472801
40997,Numto,Numto,63.6667,71.3333,Russia,RU,RUS,Khanty-Mansiyskiy Avtonomnyy Okrug-Yugra,NaN,10.0,1643985006
40998,Nord,Nord,81.7166,-17.8000,Greenland,GL,GRL,Sermersooq,NaN,10.0,1304217709
40999,Timmiarmiut,Timmiarmiut,62.5333,-42.2167,Greenland,GL,GRL,Kujalleq,NaN,10.0,1304206491


In [ ]:
 names = cities_df['city'].sample(25000)
names

10712    Villingen-Schwenningen
11678                     Rafḩā
28219                     Pinto
22404                     Piera
5490                 Tomigusuku
                  ...          
32129                Kenilworth
2501                     Garoua
9453                   Tallaght
19485               Palm Valley
36147                Seloncourt
Name: city, Length: 25000, dtype: object

## generate hotels

In [ ]:
newdf = pd.DataFrame(columns=['name','longitude','latitude'])
newdf

,name,longitude,latitude


In [ ]:
tries = 1
for name in names:
  srch = name + " hotel"
  json = make_request(srch, tries).json()
  try:
      json.get('features')[0].get('geometry').get('coordinates')
  except (KeyError, IndexError ):
      print('cannot get ' + srch)
  else:
    lon, lat = json.get('features')[0].get('geometry').get('coordinates')
    name = json['features'][0].get('properties').get('name')
    newdf.loc[len(newdf.index)] = [name, lon, lat]



cannot get Rivolta d’Adda hotel
cannot get Qasbat Tadla hotel
cannot get ’Aïn Arnat hotel
cannot get Monteroni d’Arbia hotel
cannot get Terrasini Favarotta hotel
cannot get Rignano sull’Arno hotel
cannot get Puréparo de Echaíz hotel
cannot get Quartu Sant’Elena hotel
cannot get Pallazzolo sull’Oglio hotel
cannot get Tabernes de Valldigna hotel
cannot get Mosciano Sant’Angelo hotel
cannot get Santa Margarita de Mombúy hotel
cannot get Cournon-d’Auvergne hotel
cannot get Callosa de Ensarriá hotel
cannot get Zawiat Moulay Brahim hotel
cannot get Cava de’ Tirreni hotel
cannot get Yan’an Beilu hotel
cannot get Guernica y Luno hotel
cannot get L’Isle-d’Espagnac hotel


KeyboardInterrupt: ignored

In [ ]:
newdf


,name,longitude,latitude
0,Villingen-Schwenningen,8.453180,48.064962
1,Rafha,43.494802,29.620219
2,Egido Villa de Pinto Hotel,-3.696101,40.242937
3,Hotel zia Piera,9.317917,44.316492
4,Tomigusuku,127.681920,26.185830
...,...,...,...
7742,Wadsley Church,-1.517748,53.412080
7743,Southwick,-72.770370,42.054820
7744,Hotel La Libertad,-91.502253,14.533274
7745,Cornelia Ida / Stewartville,-58.283788,6.837000


In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [ ]:
newdf.to_csv(path_or_buf='gdrive/My Drive/generatedhoteldf.csv')